# Retrieve all Datasets for Pharmacy Desert Analysis and Export to SQL Server

In [1]:
import cenpy
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import sqlalchemy
from sqlalchemy.types import String, Integer, Numeric, VARCHAR, BigInteger
import urllib
import sys
from tqdm import tqdm

## Datasets Required

- State FIPS codes
- Center of population at block-group level (2020)
- ACS 2020 5-year estimates: block group population
- ACS 2020 5-year estimates: housing units
- TIGER/Line block group shapefiles (2020)
- Relationship file for 2020 block group to 2010 block group (may use to link ADI data)

## Get State FIPS from Census Website

In [6]:
states_url = 'https://www2.census.gov/geo/docs/reference/state.txt'
col_names = ['state_fips', 'usps_code', 'state_name']
state_codes = pd.read_csv(states_url, sep='|', header=0, names=col_names, 
                          dtype=str, usecols=[0,1,2])
state_codes.head()

,state_fips,usps_code,state_name
0,01,AL,Alabama
1,02,AK,Alaska
2,04,AZ,Arizona
3,05,AR,Arkansas
4,06,CA,California


In [3]:
state_codes[50:]

,state_fips,usps_code,state_name
50,56,WY,Wyoming
51,60,AS,American Samoa
52,66,GU,Guam
53,69,MP,Northern Mariana Islands
54,72,PR,Puerto Rico
55,74,UM,U.S. Minor Outlying Islands
56,78,VI,U.S. Virgin Islands


FIPS > 56 are not areas I will analyze. 

In [7]:
# Keep codes only for 50 states + DC
state_codes = state_codes[:51]
state_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state_fips  51 non-null     object
 1   usps_code   51 non-null     object
 2   state_name  51 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [8]:
state_codes

,state_fips,usps_code,state_name
0,01,AL,Alabama
1,02,AK,Alaska
2,04,AZ,Arizona
3,05,AR,Arkansas
4,06,CA,California
5,08,CO,Colorado
6,09,CT,Connecticut
7,10,DE,Delaware
8,11,DC,District of Columbia
9,12,FL,Florida


### Export to SQL Server

In [13]:
sql_types = {'state_fips': String(length=5), 
             'usps_code': String(length=2),
             'state_name': String(length=30)}

In [14]:
# Connect to prxsql2019ml
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=prxsql2019ml;DATABASE=PublicHealth")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

# Export data
state_codes.to_sql('StateFips', schema='Census', con=engine, chunksize=200, 
                   method='multi', dtype=sql_types,
                   index=False, if_exists='replace')

51

## Block Group Relationship Files for 2020 vs 2010

In [7]:
rel_url = 'https://www2.census.gov/geo/docs/maps-data/data/rel2020/blkgrp/tab20_blkgrp20_blkgrp10_natl.txt'
col_names = ['geoid_2020', 'geoid_2010', 'land_area_2010', 'water_area_2010',
             'land_area_overlap', 'water_area_overlap']
bg_relationships = pd.read_csv(rel_url, sep='|', header=0, names=col_names,
                               dtype=str, usecols=[1,8,10,11,14,15])
print(bg_relationships.info())
bg_relationships.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337326 entries, 0 to 337325
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   geoid_2020          337326 non-null  object
 1   geoid_2010          337326 non-null  object
 2   land_area_2010      337326 non-null  object
 3   water_area_2010     337326 non-null  object
 4   land_area_overlap   337326 non-null  object
 5   water_area_overlap  337326 non-null  object
dtypes: object(6)
memory usage: 15.4+ MB
None


,geoid_2020,geoid_2010,land_area_2010,water_area_2010,land_area_overlap,water_area_overlap
0,010010201001,010010201001,4259443,28435,4259443,28435
1,010010201001,010010202002,1267300,5669,4856,0
2,010010201002,010010201002,5567828,0,5561005,0
3,010010202001,010010202001,2058374,0,2058374,0
4,010010202002,010010202002,1267300,5669,1262444,5669


In [6]:
# Figure out which cols I need to keep
for idx, col in enumerate(bg_relationships.columns.tolist()):
    print(idx, col)

0 OID_BLKGRP_20
1 GEOID_BLKGRP_20
2 NAMELSAD_BLKGRP_20
3 AREALAND_BLKGRP_20
4 AREAWATER_BLKGRP_20
5 MTFCC_BLKGRP_20
6 FUNCSTAT_BLKGRP_20
7 OID_BLKGRP_10
8 GEOID_BLKGRP_10
9 NAMELSAD_BLKGRP_10
10 AREALAND_BLKGRP_10
11 AREAWATER_BLKGRP_10
12 MTFCC_BLKGRP_10
13 FUNCSTAT_BLKGRP_10
14 AREALAND_PART
15 AREAWATER_PART


In [12]:
# Change data types
convert_cols = ['land_area_2010', 'water_area_2010', 'land_area_overlap',
                'water_area_overlap']
bg_relationships[convert_cols] = bg_relationships[convert_cols].astype('int64')
bg_relationships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337326 entries, 0 to 337325
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   geoid_2020          337326 non-null  object
 1   geoid_2010          337326 non-null  object
 2   land_area_2010      337326 non-null  int64 
 3   water_area_2010     337326 non-null  int64 
 4   land_area_overlap   337326 non-null  int64 
 5   water_area_overlap  337326 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 15.4+ MB


### Export to SQL Server

In [13]:
sql_types = {'geoid_2020':String(12),
             'geoid_2010':String(12),
             'land_area_2010':BigInteger(),
             'water_area_2010':BigInteger(),
             'land_area_overlap':BigInteger(),
             'water_area_overlap':BigInteger()}

In [14]:
# Connect to prxsql2019ml
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=prxsql2019ml;DATABASE=PublicHealth")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

# Export data
bg_relationships.to_sql('BlockGroupRelationship_2020_2010', schema='Census', 
                        con=engine, chunksize=200, method='multi', 
                        dtype=sql_types, index=False, if_exists='replace')

337326

In [8]:
bg_relationships.columns.to_list()

['geoid_2020',
 'geoid_2010',
 'land_area_2010',
 'water_area_2010',
 'land_area_overlap',
 'water_area_overlap']

## Get Census Datasets for each State, Merge, and Export to SQL Server

### Get Block Group Population Centroids from Census Website

In [2]:
def get_center_pop(state_fips):
    """
    Get the Center of Population Dataset at the block group level from 
    Census URL given a state's FIPS code
    """
    # Retrieve data
    url = 'https://www2.census.gov/geo/docs/reference/cenpop2020/blkgrp/CenPop2020_Mean_BG' \
                        + state_fips + '.txt'
    col_names = ['state_fips', 'county_fips', 'tract_fips', 'bg_fips', 
                 'center_pop_lat', 'center_pop_lon']
    data = pd.read_csv(url, header=0, names=col_names,
                       dtype=str, usecols=[0,1,2,3,5,6])
    
    # Create GEOID column
    data['geoid'] = data['state_fips'] + data['county_fips'] \
                    + data['tract_fips'] + data['bg_fips']
    
    # Conver dtype of lat, lon cols
    data[['center_pop_lat', 'center_pop_lon']] = data[['center_pop_lat', 'center_pop_lon']].astype(float)
    
    return data[['geoid', 'center_pop_lat', 'center_pop_lon']]

In [180]:
# Get data
center_pop = get_center_pop('04')
center_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   geoid           4773 non-null   object 
 1   center_pop_lat  4773 non-null   float64
 2   center_pop_lon  4773 non-null   float64
dtypes: float64(2), object(1)
memory usage: 112.0+ KB


In [181]:
center_pop.head()

,geoid,center_pop_lat,center_pop_lon
0,040019426001,36.683886,-109.841432
1,040019426002,36.865721,-109.848973
2,040019427001,36.918849,-109.128570
3,040019427002,36.777107,-109.575742
4,040019427003,36.839180,-109.393777


### Get ACS 2020 Population & Housing Units Datasets

- B01003_001E: Population estimates
- B01003_001M: Margin of error for population estimates
- B25001_001E: Housing unit estimates
- B25001_001M: Margin of error for housing unit estimates

In [3]:
# Connect to API
api_conn = cenpy.remote.APIConnection('ACSDT5Y2020')
api_conn

Connection to American Community Survey: 5-Year Estimates: Detailed Tables 5-Year (ID: https://api.census.gov/data/id/ACSDT5Y2020)

In [12]:
def get_acs_datasets(state_fips):
    """
    Given a state's FIPS code, retrieve ACS 2020 5-year estimates for 
    block group population and housing units using the cenpy library
    """
    # Define data to retrieve
    variables = ['GEO_ID', 
                'B01003_001E', 'B01003_001M', # Population
                'B25001_001E', 'B25001_001M'] # Housing units
    
    # Retrieve data
    data = api_conn.query(variables, 
                          geo_unit="block group:*",
                          geo_filter={'state':state_fips, 
                                      'county':'*',
                                      'tract':'*'})
    # Rename columns
    data = data.rename(columns={'state':'state_fips',
                                'B01003_001E':'population', 
                                'B01003_001M':'pop_moe',
                                'B25001_001E':'housing_units', 
                                'B25001_001M':'housing_moe'})
    
    # Fix geoid values
    data['geoid'] = data['GEO_ID'].str[-12:]
    
    # Change data types
    data[['population', 'pop_moe', 'housing_units', 'housing_moe']] = data[['population', 'pop_moe','housing_units', 'housing_moe']].astype(int)
    
    return data[['geoid', 'population', 'pop_moe', 
                 'housing_units', 'housing_moe']]

In [182]:
acs = get_acs_datasets('04')
acs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   geoid          4773 non-null   object
 1   population     4773 non-null   int32 
 2   pop_moe        4773 non-null   int32 
 3   housing_units  4773 non-null   int32 
 4   housing_moe    4773 non-null   int32 
dtypes: int32(4), object(1)
memory usage: 112.0+ KB


In [153]:
acs.head()

,geoid,population,pop_moe,housing_units,housing_moe
0,010610505004,1137,384,522,121
1,010610503006,758,339,472,230
2,010610504003,678,243,438,153
3,010610505001,787,195,398,83
4,010610506003,1591,347,663,127


### Import TIGER/Line Shapefile

Downloaded via FTP as zip files

In [13]:
def get_shapefile(state_fips):
    """
    Import block group shapefiles into a GeoDataFrame from appropriate zip
    file given a state's FIPS code and return data as DataFrame.
    """
    # Import data
    zip_path = 'zip://Data/TIGERLineShapefiles/tl_2020_' + state_fips + '_bg.zip'
    df = pd.DataFrame(gpd.read_file(zip_path)) # Use geopandas to import geometry but convert to dataframe
    
    # Rename columns
    df = df.rename(columns={'STATEFP': 'state_fips',
                            'COUNTYFP': 'county_fips',
                            'TRACTCE': 'tract_fips',
                            'BLKGRPCE': 'bg_fips',
                            'GEOID': 'geoid',
                            'ALAND':'land_area',
                            'AWATER': 'water_area', 
                            'INTPTLAT': 'centroid_lat', 
                            'INTPTLON': 'centroid_lon', 
                            'geometry': 'boundaries'})

    # Change data types for lat, lon
    df[['centroid_lat', 'centroid_lon']] = df[['centroid_lat', 'centroid_lon']].astype(float)
    df['boundaries'] = df['boundaries'].astype(str)

    # Colums to return
    cols = ['geoid', 'state_fips', 'county_fips', 'tract_fips', 'bg_fips',
            'centroid_lat','centroid_lon','land_area','water_area',
            'boundaries']
    
    return df[cols]

In [183]:
shp = get_shapefile('04')
shp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   geoid         4773 non-null   object 
 1   state_fips    4773 non-null   object 
 2   county_fips   4773 non-null   object 
 3   tract_fips    4773 non-null   object 
 4   bg_fips       4773 non-null   object 
 5   centroid_lat  4773 non-null   float64
 6   centroid_lon  4773 non-null   float64
 7   land_area     4773 non-null   int64  
 8   water_area    4773 non-null   int64  
 9   boundaries    4773 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 373.0+ KB


In [77]:
shp.head()

,geoid,state_fips,county_fips,tract_fips,bg_fips,centroid_lat,centroid_lon,land_area,water_area,boundaries
0,220790125001,22,079,012500,1,31.260977,-92.460768,1732153,0,"POLYGON ((-92.47676 31.25906, -92.47660 31.259..."
1,220330010003,22,033,001000,3,30.458868,-91.153126,496094,0,"POLYGON ((-91.15540 30.46422, -91.15540 30.464..."
2,220330035053,22,033,003505,3,30.491510,-91.114284,1883557,0,"POLYGON ((-91.12614 30.49574, -91.12537 30.495..."
3,220330035054,22,033,003505,4,30.483303,-91.104274,1897809,0,"POLYGON ((-91.11880 30.48511, -91.11859 30.485..."
4,220330035061,22,033,003506,1,30.485563,-91.063814,1068709,0,"POLYGON ((-91.06973 30.48578, -91.06948 30.485..."


### Merge Datasets & Export to SQL Server

In [184]:
# Define data types for table in SQL Server
sql_types = {'geoid': String(12),
             'state_fips': String(5),
             'county_fips': String(5),
             'tract_fips': String(10),
             'bg_fips': String(2),
             'centroid_lat': Numeric(15,7),
             'centroid_lon': Numeric(15,7),
             'land_area': BigInteger(),
             'water_area': BigInteger(),
             'boundaries': VARCHAR(None),
             'center_pop_lat': Numeric(15,7),
             'center_pop_lon': Numeric(15,7),
             'population': Integer(),
             'pop_moe': Integer(),
             'housing_units': Integer(),
             'housing_moe': Integer()}

In [185]:
# Connect to prxsql2019ml.PublicHealth
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=prxsql2019ml;DATABASE=PublicHealth")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [28]:
def merge_and_export(state_fips):
    # Use left joins in case mismatch in # of records between datasets
    block_group = pd.merge(pd.merge(shp, center_pop, how='left'), acs, how='left')
    
    # Export data
    block_group.to_sql('BlockGroup', schema='Census', con=engine, 
                       chunksize=100, method='multi', dtype=sql_types,
                       index=False, if_exists='append')
    print(f'Exported dataset for state # {state_fips} to SQL Server.\n')

In [187]:
merge_and_export('04')

Exported dataset for state # 04 to SQL Server.


## Combine functions to run in a loop

In [10]:
# Get list of state FIPS codes
state_fips_list = state_codes['state_fips'].to_list()
len(state_fips_list)

51

In [15]:
# Connect to API
api_conn = cenpy.remote.APIConnection('ACSDT5Y2020')
api_conn

Connection to American Community Survey: 5-Year Estimates: Detailed Tables 5-Year (ID: https://api.census.gov/data/id/ACSDT5Y2020)

In [16]:
# Define data types for table in SQL Server
sql_types = {'geoid': String(12),
             'state_fips': String(5),
             'county_fips': String(5),
             'tract_fips': String(10),
             'bg_fips': String(2),
             'centroid_lat': Numeric(15,7),
             'centroid_lon': Numeric(15,7),
             'land_area': BigInteger(),
             'water_area': BigInteger(),
             'boundaries': VARCHAR(None),
             'center_pop_lat': Numeric(15,7),
             'center_pop_lon': Numeric(15,7),
             'population': Integer(),
             'pop_moe': Integer(),
             'housing_units': Integer(),
             'housing_moe': Integer()}

In [17]:
# Connect to prxsql2019ml.PublicHealth
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=prxsql2019ml;DATABASE=PublicHealth")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [29]:
for fips in tqdm(state_fips_list):
    try:
        # Retrieve datasets
        center_pop = get_center_pop(fips)
        acs = get_acs_datasets(fips)
        shp = get_shapefile(fips)
        
        # Get row number for each dataset
        center_pop_nrow = center_pop.shape[0]
        acs_nrow = acs.shape[0]
        shp_nrow = shp.shape[0]
        
        # Check if datasets have same # of block groups
        if center_pop_nrow == acs_nrow == shp_nrow:
            pass
        else:
            print(f"State # {fips} has mismatch in block group number between datasets.")
            print(f"center_pop row #: {center_pop_nrow}")
            print(f"acs row #: {acs_nrow}")
            print(f"shapefiles row #: {shp_nrow}")
            print()
        
        # Get final dataset and export to SQL server
        merge_and_export(fips)
    except:
        print("Oops! ", sys.exc_info()[0:2])
        print()

  2%|█▋                                                                                 | 1/51 [00:19<16:10, 19.42s/it]

Exported dataset for state # 01 to SQL Server.



  4%|███▎                                                                               | 2/51 [00:24<09:11, 11.26s/it]

Exported dataset for state # 02 to SQL Server.



  6%|████▉                                                                              | 3/51 [00:37<09:38, 12.05s/it]

...

Exported dataset for state # 54 to SQL Server.



 98%|████████████████████████████████████████████████████████████████████████████████▍ | 50/51 [15:19<00:19, 19.43s/it]

Exported dataset for state # 55 to SQL Server.



100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [15:23<00:00, 18.11s/it]

Exported dataset for state # 56 to SQL Server.

